In [7]:
import urllib.request, urllib.parse, urllib.error
import http
import sqlite3
import json
import time
import ssl
import sys

api_key = False
# If you have a Google Places API key, enter it here
# api_key = 'AIzaSy___IDByT70'

if api_key is False:
    api_key = 42
    serviceurl = "http://py4e-data.dr-chuck.net/json?"
else :
    serviceurl = "https://maps.googleapis.com/maps/api/geocode/json?"

# Additional detail for urllib
# http.client.HTTPConnection.debuglevel = 1

conn = sqlite3.connect('geodata.sqlite')
cur = conn.cursor()

cur.execute('''
CREATE TABLE IF NOT EXISTS Locations (address TEXT, geodata TEXT)''')

# Ignore SSL certificate errors
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

fh = open("where.data")
count = 0
for line in fh:
    if count > 200 :
        print('Retrieved 200 locations, restart to retrieve more')
        break

    address = line.strip()
    print('')
    cur.execute("SELECT geodata FROM Locations WHERE address= ?",
        (memoryview(address.encode()), ))

    try:
        data = cur.fetchone()[0]
        print("Found in database ",address)
        continue
    except:
        pass

    parms = dict()
    parms["address"] = address
    if api_key is not False: parms['key'] = api_key
    url = serviceurl + urllib.parse.urlencode(parms)

    print('Retrieving', url)
    uh = urllib.request.urlopen(url, context=ctx)
    data = uh.read().decode()
    print('Retrieved', len(data), 'characters', data[:20].replace('\n', ' '))
    count = count + 1

    try:
        js = json.loads(data)
    except:
        print(data)  # We print in case unicode causes an error
        continue

    if 'status' not in js or (js['status'] != 'OK' and js['status'] != 'ZERO_RESULTS') :
        print('==== Failure To Retrieve ====')
        print(data)
        break

    cur.execute('''INSERT INTO Locations (address, geodata)
            VALUES ( ?, ? )''', (memoryview(address.encode()), memoryview(data.encode()) ) )
    conn.commit()
    if count % 10 == 0 :
        print('Pausing for a bit...')
        time.sleep(5)

print("Run geodump.py to read the data from the database so you can vizualize it on a map.")


Found in database  AGH University of Science and Technology

Found in database  Academy of Fine Arts Warsaw Poland

Found in database  American University in Cairo

Found in database  Arizona State University

Found in database  Athens Information Technology

Retrieving http://py4e-data.dr-chuck.net/json?address=ISI+Kolkata&key=42
Retrieved 2936 characters {    "results" : [  

Retrieving http://py4e-data.dr-chuck.net/json?address=NIT+Jamshedpur&key=42
Retrieved 2026 characters {    "results" : [  

Retrieving http://py4e-data.dr-chuck.net/json?address=Capgemini+Mumbai&key=42
Retrieved 2592 characters {    "results" : [  

Found in database  BIT Mesra

Found in database  Babcock University

Found in database  Banaras Hindu University

Found in database  Bangalore University

Found in database  Baylor University

Found in database  Beijing normal university

Found in database  Belarusian State University

Found in database  Belgrade University
Run geodump.py to read the data from the d

In [8]:
import sqlite3
import json
import codecs

conn = sqlite3.connect('geodata.sqlite')
cur = conn.cursor()

cur.execute('SELECT * FROM Locations')
fhand = codecs.open('where.js', 'w', "utf-8")
fhand.write("myData = [\n")
count = 0
for row in cur :
    data = str(row[1].decode())
    try: js = json.loads(str(data))
    except: continue

    if not('status' in js and js['status'] == 'OK') : continue

    lat = js["results"][0]["geometry"]["location"]["lat"]
    lng = js["results"][0]["geometry"]["location"]["lng"]
    if lat == 0 or lng == 0 : continue
    where = js['results'][0]['formatted_address']
    where = where.replace("'", "")
    try :
        print(where, lat, lng)

        count = count + 1
        if count > 1 : fhand.write(",\n")
        output = "["+str(lat)+","+str(lng)+", '"+where+"']"
        fhand.write(output)
    except:
        continue

fhand.write("\n];\n")
cur.close()
fhand.close()
print(count, "records written to where.js")
print("Open where.html to view the data in a browser")


aleja Adama Mickiewicza 30, 30-059 Kraków, Poland 50.06688579999999 19.9136192
Krakowskie Przedmieście 5, 00-068 Warszawa, Poland 52.2394019 21.0150792
AUC Avenue، Road، First New Cairo, Cairo Governorate 11835, Egypt 30.0189275 31.499707
Phoenix, AZ 85004, USA 33.4532165 -112.0719833
Athens, OH 45701, USA 39.3301047 -82.1076022
Vidya Vihar, Pilani, Rajasthan 333031, India 28.3588163 75.58802039999999
Ilishan-Remo, Nigeria 6.8939569 3.7187158
Ajagara, Varanasi, Uttar Pradesh 221005, India 25.2677203 82.99125819999999
Mysore Rd, Jnana Bharathi, Bengaluru, Karnataka 560056, India 12.9503878 77.5022224
1311 S 5th St, Waco, TX 76706, USA 31.5488923 -97.1130573
19 Xinjiekou Outer St, Bei Tai Ping Zhuang, Haidian Qu, Beijing Shi, China, 100875 39.9619537 116.3662615
praspiekt Niezaliežnasci 4, Minsk, Belarus 53.8938988 27.5460609
Studentski trg 1, Beograd, Serbia 44.8184518 20.4575913
700 College St, Beloit, WI 53511, USA 42.5030333 -89.0309048
praspiekt Niezaliežnasci 4, Minsk, Belarus 53.8

Avenida Pedro Calmon, 550 Reitoria - Cidade Universitária da Universidade Federal do Rio de Janeiro, Rio de Janeiro - RJ, 21941-901, Brazil -22.8625345 -43.2234737
Av. Rovisco Pais 1, 1049-001 Lisboa, Portugal 38.7368192 -9.138705
Butanta, São Paulo - State of São Paulo, Brazil -23.5613991 -46.7307891
R. da Universidade, 4710-057 Braga, Portugal 41.5607319 -8.3962368
Bulaksumur, Caturtunggal, Kec. Depok, Kabupaten Sleman, Daerah Istimewa Yogyakarta 55281, Indonesia -7.7713847 110.3774998
Kolkata, West Bengal, India 22.572646 88.36389500000001
Jamshedpur, Jharkhand, India 22.8045665 86.2028754
Mumbai, Maharashtra, India 19.0759837 72.8776559
Mesra, Jharkhand 835215, India 23.412305 85.43990099999999
Plot No, 203, Barrackpore Trunk Rd, Dunlop, Bonhooghly Government Colony, Baranagar, Kolkata, West Bengal 700108, India 22.648777 88.377011
Adityapur, Jamshedpur, Jharkhand 831014, India 22.7770174 86.144116
M6, Capgemini Knowledge Park, Off, Thane - Belapur Rd, TTC Industrial Area, Airoli, 

In [9]:
ord('G')

71

In [11]:
chr(108)

'l'

In [12]:
chr(105)

'i'

In [13]:
chr(115)

's'

In [14]:
chr(116)

't'

In [15]:
zap = "hello there bob"
print(zap[4])

o


In [16]:
x = 12
if x < 5:
print("smaller")
else:
    print("bigger")
print("all done")

IndentationError: expected an indented block (<ipython-input-16-adfc3d2d2a4c>, line 3)

In [17]:
total = 0
for abc in range(5):
    total = total + abc
print(total)


10


In [18]:
a = "123"
b = 456
c = a + b
print(c)

TypeError: can only concatenate str (not "int") to str

In [19]:
abc = "With three words"
stuff = abc.split()
print(stuff)

['With', 'three', 'words']
